# 11. GRAD SCHOOL ADMISSIONS: EVALUATING BINARY CLASSIFIERS
---

## 1. Introducing the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

admissions = pd.read_csv('admissions.csv')
train, test = train_test_split(admissions, test_size=0.2, random_state=42)
print(train.shape, test.shape)

(515, 3) (129, 3)


In [2]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train[["gpa"]], train["admit"])

labels = model.predict(train[["gpa"]])
train_ = train.copy()
train_["predicted_label"] = labels
train_["predicted_label"].value_counts()

0    403
1    112
Name: predicted_label, dtype: int64

In [3]:
train_.head()

,admit,gpa,gre,predicted_label
515,1,3.606,570.618,1
29,0,2.783,585.700,0
274,0,3.227,450.289,0
576,1,3.540,615.932,1
434,1,3.743,593.282,1


## 2. Accuracy

$$
Accuracy = \frac{n Correctly-Predicted}{n Observations}
$$

In [4]:
train_["actual_label"] = train_["admit"]
matches = train_["predicted_label"] == train_["actual_label"]
correct_predictions = train_[matches]
correct_predictions.head()

,admit,gpa,gre,predicted_label,actual_label
515,1,3.606,570.618,1,1
29,0,2.783,585.700,0,0
274,0,3.227,450.289,0,0
576,1,3.540,615.932,1,1
434,1,3.743,593.282,1,1


In [5]:
accuracy = len(correct_predictions) / len(admissions)
print(accuracy)

0.5388198757763976


## 3. Binary classification outcomes
- `True Positive`: the model predicted that the label would be `Positive`, and that ended up being `True`
- `True Negative`: the model predicted that the label would be `Negative`, and that ended up being `True`
- `False Positive`: the model predicted that the label would be `Positive`, but that was `False` (the actual label was False)
- `False Negative`: the model predicted that the label would be `Negative`, but that was `False` (the actual value was True)

In [6]:
TP_filter = (train_["predicted_label"] == 1) & (train_["actual_label"] == 1)
TP = len(train_[TP_filter])

TN_filter = (train_["predicted_label"] == 0) & (train_["actual_label"] == 0)
TN = len(train_[TN_filter])

print(TP)
print(TN)

73
274


## 4. Sensitivity
- `True Positive Rate` - The proportion of applicants that were correctly admitted:
$$
TPR = \frac{TP}{TP + FN}
$$
- How effective is this model at identifying positive outcomes?

In [7]:
TP_filter = (train_["predicted_label"] == 1) & (train_["actual_label"] == 1)
TP = len(train_[TP_filter])

FN_filter = (train_["predicted_label"] == 0) & (train_["actual_label"] == 1)
FN = len(train_[FN_filter])

sensitivity = TP / (TP + FN)
print(sensitivity)

0.3613861386138614


Looks like the sensitivity of the model is around 36.1% and only about 1 in 8 students that should have been admitted were actually admitted. In the context of predicting student admissions, this probably isn't too bad of a thing. Graduate schools can only admit a select number of students into their programs and by definition they end up rejecting many qualified students that would have succeeded.

In the healthcare context, however, low sensitivity could mean a severe loss of life. If a classification model is only catching 36.1% of positive cases for an illness, then around 7 of 8 people are going undiagnosed (being classified as false negatives).
## 5. Specificity
- `True Negative Rate` - The proportion of applicants that were correctly rejected:
$$
TPR = \frac{TP}{TP + FN}
$$
- How effective is this model at identifying negative outcomes?

In [8]:
TN_filter = (train_["predicted_label"] == 0) & (train_["actual_label"] == 0)
TN = len(train_[TN_filter])

FP_filter = (train_["predicted_label"] == 1) & (train_["actual_label"] == 0)
FP = len(train_[FP_filter])

specificity = TN / (FP + TN)
print(specificity)

0.8753993610223643


It looks like the specificity of the model is 87.5%. This means that the model is really good at knowing which applicants to reject. Since a small number of applicants were accepted that applied, it's important that the model reject people correctly who wouldn't have otherwise been accepted.